# Analisando Pacientes e Riscos

## imports

In [ ]:
!pip install ipdb
!pip install tqdm
from tqdm import tqdm
import pandas as pd
from collections import ChainMap

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import importlib
from src import bases_utils as bu
importlib.reload(bu)

<module 'src.bases_utils' from 'c:\\Users\\thiag\\OneDrive\\Documents\\UFRJ\\git\\projeto_lupus\\src\\bases_utils.py'>

## Leitura das Bases

### Procedimentos com Risco

In [3]:
df_risk = pd.read_csv('data/procedimentos/dev/procedimentos_encontrados_no_risco_cirurgico.csv')
set_risk = set(df_risk['cod_procedimento'])

### Com Lúpus

In [4]:
num_algoritmo = 2

df_lupus = pd.read_parquet(f'data/results/df_alg{num_algoritmo}.parquet')
df_lupus_bpai = pd.read_parquet(f'data/results/df_bpai_alg{num_algoritmo}.parquet')
df_lupus_aih = pd.read_parquet(f'data/results/df_aih_alg{num_algoritmo}.parquet')
df_lupus_apac = pd.read_parquet(f'data/results/df_apac_alg{num_algoritmo}.parquet')

### Sem Lúpus

In [130]:
df_aih = pd.read_parquet(f'data/AIH_Lupus_L93M32N08_todos_cids.parquet')

In [5]:
df_bpai = pd.read_parquet(f'data/BPAI_Lupus_L93M32N08_todos_cids.parquet')
df_aih = pd.read_parquet(f'data/AIH_Lupus_L93M32N08_todos_cids.parquet')
df_apac = pd.read_parquet(f'data/APAC_Lupus_L93M32N08_todos_cids.parquet')

# Preciso ajustas as datas
df_aih['data'] = pd.to_datetime(df_aih['dt_internacao'], format='%d/%m/%Y', errors='coerce')
df_apac['data'] = pd.to_datetime(df_apac['data_inicio'], format='%d/%m/%Y', errors='coerce')

# Não achei data na bpai
# Só para conseguir unificar o bpai
df_bpai['data'] = '01/01/2058'
df_bpai['data'] = pd.to_datetime(df_bpai['data'], format='%d/%m/%Y', errors='coerce')

In [6]:
def get_cols_procedimentos(base_name):
      dict_col = {'BPAI':['co_procedimento_realizado'],
                  'AIH':['co_procedimento_principal'], # --> AIH nãm tem cod_procedimento_secundario
                  'APAC':['co_procedimento_principal', 'co_procedimento_secundario'],
                  'UNION':['co_procedimento_realizado', # BPAI
                           'co_procedimento_principal', # APAC
                           'co_procedimento_secundario', # APAC
                           'co_procedimento_aih']} # AIH
      return dict_col[base_name]

def get_risk(df, cols, set_risk):
    """ Calcula quantos pacientes possuem um procedimento de risco """
    if len(cols) > 1: # Se tiver mais de uma coluna de procedimentos
        cols_risk = []
        for idx, col in enumerate(cols): # Itera nas colunas
            col_name = f'risk_procedimento_{idx}'
            df[col_name] = df[col].isin(set_risk)
            cols_risk.append(col_name)

        df['risk_filter_col'] = df[cols_risk].sum(axis=1) # Conta quantas colnuas possuem True
        
        # Filtra as que tiveram pelo menos 1
        num_pacientes_risk = df[df['risk_filter_col'] > 0]['id_paciente'].nunique()

    else: # Se for só uma coluna, fica mais fácil
        num_pacientes_risk = df[df[cols[0]].isin(set_risk)]['id_paciente'].nunique()
    return num_pacientes_risk

def get_percent(numerador, denominador):
    return round(numerador*100/denominador, 2)

def analisa_riscos(df, df_lupus, base_name):
    cols_procedimento = get_cols_procedimentos(base_name)

    num_total_pacientes = df['id_paciente'].nunique()
    print(f' - Pacientes na {base_name}:', num_total_pacientes)
    
    num_pacientes_risk = get_risk(df, cols_procedimento, set_risk)
    print(f' - Pacientes na {base_name} com algum risco: '+
            f'{num_pacientes_risk} ' +
            f'({get_percent(num_pacientes_risk, num_total_pacientes)}%)')

    num_pacientes_lupus = df_lupus['id_paciente'].nunique()
    print(f' - Pacientes com Lúpus na {base_name}: {num_pacientes_lupus} '+
          f'({get_percent(num_pacientes_lupus, num_total_pacientes)})%')

    num_lupus_risk = get_risk(df_lupus, cols_procedimento, set_risk)

    print(f' - Pacientes com Lúpus e algum risco na {base_name}: {num_lupus_risk} ' +
            f'({get_percent(num_lupus_risk, num_pacientes_lupus)}% de Lúpus) e ' +
            f'({get_percent(num_lupus_risk, num_total_pacientes)}% da {base_name})')
    print()

analisa_riscos(df_bpai, df_lupus_bpai, 'BPAI')
analisa_riscos(df_aih, df_lupus_aih, 'AIH')
analisa_riscos(df_apac, df_lupus_apac, 'APAC')

 - Pacientes na BPAI: 46967
 - Pacientes na BPAI com algum risco: 33049 (70.37%)
 - Pacientes com Lúpus na BPAI: 19349 (41.2)%
 - Pacientes com Lúpus e algum risco na BPAI: 15548 (80.36% de Lúpus) e (33.1% da BPAI)

 - Pacientes na AIH: 48663
 - Pacientes na AIH com algum risco: 6906 (14.19%)
 - Pacientes com Lúpus na AIH: 30451 (62.58)%
 - Pacientes com Lúpus e algum risco na AIH: 4601 (15.11% de Lúpus) e (9.45% da AIH)

 - Pacientes na APAC: 31948
 - Pacientes na APAC com algum risco: 14418 (45.13%)
 - Pacientes com Lúpus na APAC: 21906 (68.57)%
 - Pacientes com Lúpus e algum risco na APAC: 6942 (31.69% de Lúpus) e (21.73% da APAC)



# Base Unificada
BPAI + AIH + APAC

In [7]:
df_alg = pd.read_parquet(f'data/results/df_alg{num_algoritmo}.parquet')

In [8]:
cols_union = ['id_paciente'
              #, 'sexo', 'idade', 'data'
              ]
df_aih.rename(columns={'co_procedimento_principal':'co_procedimento_aih'})
df_union = pd.concat([df_bpai[cols_union+get_cols_procedimentos('BPAI')],
                      df_apac[cols_union+get_cols_procedimentos('APAC')],
                      df_aih.rename(columns={'co_procedimento_principal':'co_procedimento_aih'})\
                      [cols_union+['co_procedimento_aih']]])

In [9]:
cols_union = ['id_paciente']
df_lupus_aih.rename(columns={'co_procedimento_principal':'co_procedimento_aih'})
df_lupus_union = pd.concat([df_lupus_bpai[cols_union+get_cols_procedimentos('BPAI')],
                      df_lupus_apac[cols_union+get_cols_procedimentos('APAC')],
                      df_lupus_aih.rename(columns={'co_procedimento_principal':'co_procedimento_aih'})\
                      [cols_union+['co_procedimento_aih']]])

In [10]:
analisa_riscos(df_union, df_lupus_union, 'UNION')

 - Pacientes na UNION: 71874
 - Pacientes na UNION com algum risco: 43032 (59.87%)
 - Pacientes com Lúpus na UNION: 31583 (43.94)%
 - Pacientes com Lúpus e algum risco na UNION: 19634 (62.17% de Lúpus) e (27.32% da UNION)



# Analise dos grupos

In [11]:
set_bpai = list(df_lupus_bpai['id_paciente'].unique())
set_aih = list(df_lupus_aih['id_paciente'].unique())
set_apac = list(df_lupus_apac['id_paciente'].unique())

df_lupus_bpai_full = df_bpai.set_index('id_paciente').loc[set_bpai].reset_index()
df_lupus_aih_full = df_aih.set_index('id_paciente').loc[set_aih].reset_index()
df_lupus_apac_full = df_apac.set_index('id_paciente').loc[set_apac].reset_index()

### Resultado Algoritmo

In [12]:
df_lupus = bu.junta_dfs(df_lupus_bpai_full, df_lupus_aih_full, df_lupus_apac_full)

### Filtro Cirurgia

In [14]:
df_cirurgia = pd.read_csv('data/procedimentos/dev/procedimentos_cirúrgicos_para_o_baseline.csv')

In [15]:
# Ajustando o nome dos procedimentos
df_union.rename(columns={'co_procedimento_realizado':'co_procedimento_bpai'}, inplace=True)

cirurgias_apac_1 = set(df_union['co_procedimento_principal']).intersection(set(df_cirurgia['cod_procedimento']))
cirurgias_apac_2 = set(df_union['co_procedimento_secundario']).intersection(set(df_cirurgia['cod_procedimento']))
print(' - Cirurgias na APAC:', len(cirurgias_apac_1)+len(cirurgias_apac_2))

cirurgias_bpai = set(df_union['co_procedimento_bpai']).intersection(set(df_cirurgia['cod_procedimento']))
print(' - Cirurgias no BPAI:', len(cirurgias_bpai))

# Verifica quem fez cirurgia
cirurgias_aih = set(df_union['co_procedimento_aih']).intersection(set(df_cirurgia['cod_procedimento']))
print(' - Cirurgias no BPAI:', len(cirurgias_aih))

 - Cirurgias na APAC: 1
 - Cirurgias no BPAI: 27
 - Cirurgias no BPAI: 191


In [16]:
# Verificando os procedimentos de cirurgia
df_union['cirurgia_aih'] = df_union['co_procedimento_aih'].isin(cirurgias_aih)
df_union['cirurgia_bpai'] = df_union['co_procedimento_bpai'].isin(cirurgias_bpai)
df_union['cirurgia_apac'] = df_union['co_procedimento_secundario'].isin(cirurgias_apac_2)
df_union['cirurgia'] = df_union['cirurgia_aih'] | df_union['cirurgia_bpai'] | df_union['cirurgia_apac']

# Pacientes que passaram por algum dos procedimentos
# cirúrgicos espefificados (em qualquer uma das bases)
aux = df_union.groupby('id_paciente')['cirurgia'].sum()
pacientes_operados = set(aux[aux>0].index)

print(' - Pacientes operados na base toda:', len(pacientes_operados))
print(f' - Pacientes operados no algoritmo {num_algoritmo}:', len(pacientes_operados.intersection(set(df_lupus['id_paciente']))))
#print(' - Pacientes operados no algoritmo 2:', len(pacientes_operados.intersection(set(df_lupus_2['id_paciente']))))

 - Pacientes operados na base toda: 9127
 - Pacientes operados no algoritmo 2: 6387


In [17]:
over_18 = df_lupus[df_lupus['idade'] >=18]
print(' -  Pacientes operados com idade >= 18', len(pacientes_operados.intersection(set(over_18['id_paciente']))))

 -  Pacientes operados com idade >= 18 6157


### Analisando os Pacientes com Lúpus e operados

In [18]:
# Questão número 1:
# Tem pacientes com mais de 1 sexo e mais de 1 região:
# Será que tenho que pegar a data e a região da cirugia? :S
count_sexo = df_lupus.groupby('id_paciente')['sexo'].nunique().sort_values()
print(count_sexo.value_counts())

count_regiao = df_lupus.groupby('id_paciente')['regiao'].nunique().sort_values()
print(count_regiao.value_counts())

count_idade = df_lupus.groupby('id_paciente')['idade_categoria'].nunique().sort_values()
print(count_idade.value_counts())

sexo
1    29769
2     1814
Name: count, dtype: int64
regiao
1    30055
2     1489
3       38
4        1
Name: count, dtype: int64
idade_categoria
1    11613
2    11318
3     7824
4      765
5       50
6       11
7        2
Name: count, dtype: int64


### Tentando pegar os dados no dia da cirurgia

In [19]:
df_lupus_cirurgia = df_lupus[df_lupus['co_procedimento_principal'].isin(df_cirurgia['cod_procedimento'])]
df_lupus_cirurgia.shape

(130659, 8)

In [20]:
set_lupus_cirurgia = df_lupus_cirurgia['id_paciente'].drop_duplicates()
df_lupus_cirurgia_idx = df_lupus_cirurgia.set_index('id_paciente')
infos_pacientes_lupus_cirurgia = set_lupus_cirurgia.apply(lambda row: bu.get_info_paciente(row, df_lupus_cirurgia_idx, df_cirurgia))
dict_aux = dict(ChainMap(*list(infos_pacientes_lupus_cirurgia)))
df_pacientes_lupus_cirugia = pd.DataFrame.from_dict(dict_aux).T
df_pacientes_lupus_cirugia['regiao'] = df_pacientes_lupus_cirugia['uf_paciente'].map(bu.get_regiao)
df_pacientes_lupus_cirugia['idade_categoria'] = df_pacientes_lupus_cirugia['idade'].map(bu.get_categoria_idade)
df_pacientes_lupus_cirugia.head()

,data,sexo,idade,uf_paciente,regiao,idade_categoria
17231471100,2018-12-09 00:00:00,FEMININO,33,PR,Sul,31-35
34899773600,2018-09-23 00:00:00,FEMININO,16,AC,Norte,16-20
9560651700,2018-11-06 00:00:00,FEMININO,71,RS,Sul,55+
19728487200,2018-12-01 00:00:00,MASCULINO,52,SP,Sudeste,51-55
89426845001,2018-09-23 00:00:00,FEMININO,16,AC,Norte,16-20


In [21]:
print(' - Pacientes com lúpus que fizeram curigia abdominal:',
      df_lupus_cirurgia['id_paciente'].nunique(), 'de',
      df_lupus['id_paciente'].nunique())

 - Pacientes com lúpus que fizeram curigia abdominal: 6386 de 31583


### Agora sim analisando os pacientes com Lúpus que operaram

In [22]:
df_pacientes_lupus_cirugia['sexo'].value_counts(normalize=True)

sexo
FEMININO     0.866427
MASCULINO    0.133573
Name: proportion, dtype: float64

In [23]:
df_pacientes_lupus_cirugia['regiao'].value_counts(normalize=True)

regiao
Sudeste         0.466019
Nordeste        0.237081
Sul             0.161290
Centro-Oeste    0.071406
Norte           0.064203
Name: proportion, dtype: float64

In [24]:
df_pacientes_lupus_cirugia['idade_categoria'].value_counts(normalize=True)

idade_categoria
21-25             0.168180
26-30             0.162543
31-35             0.135139
16-20             0.116192
55+               0.107266
36-40             0.088788
41-45             0.065925
46-50             0.055121
Idade inválida    0.052145
51-55             0.048700
Name: proportion, dtype: float64

### Testando a base do Fernando

In [25]:
tc2 = pd.read_parquet('data/pendrive/caso_passo2_com_categorias_com_riscos.parquet')
pacientes_sem_lupus = tc2[~tc2['id_paciente'].isin(df_lupus['id_paciente'])]['id_paciente']
print(' - Total de Pacientes que não foram classificados com lúpus:', len(pacientes_sem_lupus))

 - Total de Pacientes que não foram classificados com lúpus: 44


In [26]:
# Junta os 3 dfs para poder olhar todos os procedimentos de forma simplificada
df_pacientes = bu.junta_dfs_e_procedimentos(df_bpai, df_aih, df_apac)
df_pacientes.loc[df_pacientes['id_paciente']==pacientes_sem_lupus.iloc[30],
    ['id_paciente', 'no_procedimento_principal', 'no_procedimento_secundario']]

,id_paciente,no_procedimento_principal,no_procedimento_secundario
2495056,19454259800,CURETAGEM UTERINA EM MOLA HIDATIFORME,TRANSFUSAO DE PLASMA FRESCO
2502249,19454259800,CURETAGEM UTERINA EM MOLA HIDATIFORME,CURETAGEM UTERINA EM MOLA HIDATIFORME
2502730,19454259800,CURETAGEM UTERINA EM MOLA HIDATIFORME,CURETAGEM UTERINA EM MOLA HIDATIFORME
2502838,19454259800,CURETAGEM UTERINA EM MOLA HIDATIFORME,CURETAGEM UTERINA EM MOLA HIDATIFORME
2503920,19454259800,DIAGNOSTICO E/OU ATENDIMENTO DE URGENCIA EM CL...,DIAGNOSTICO E/OU ATENDIMENTO DE URGENCIA EM CL...
2504399,19454259800,DIAGNOSTICO E/OU ATENDIMENTO DE URGENCIA EM CL...,DIAGNOSTICO E/OU ATENDIMENTO DE URGENCIA EM CL...
2508612,19454259800,CURETAGEM UTERINA EM MOLA HIDATIFORME,DIÁRIA DE PERMANÊNCIA A MAIOR
2520725,19454259800,DIAGNOSTICO E/OU ATENDIMENTO DE URGENCIA EM CL...,CONSULTA/AVALIAÇÃO EM PACIENTE INTERNADO
2521116,19454259800,DIAGNOSTICO E/OU ATENDIMENTO DE URGENCIA EM CL...,CONSULTA/AVALIAÇÃO EM PACIENTE INTERNADO
6133350,19454259800,HIDROXICLOROQUINA 400 MG (POR COMPRIMIDO),HIDROXICLOROQUINA 400 MG (POR COMPRIMIDO)


#### Verifica Cids

In [27]:
def verifica_cid(row):
    for cod in ['M32', 'L93', 'N08']:
        if (cod in row):
            return True    
    return False

df_pacientes['cid_principal'] = df_pacientes['co_cid_principal'].apply(verifica_cid)
df_pacientes['cid_secundario'] = df_pacientes['co_cid_secundario'].apply(verifica_cid)

In [28]:
df_pacientes.loc[:,'cid_principal':].mean()

cid_principal     0.089116
cid_secundario    0.135328
dtype: float64

#### Verifica Procedimentos

In [29]:
import importlib
from src import bases_utils as bu
importlib.reload(bu)

<module 'src.bases_utils' from 'c:\\Users\\thiag\\OneDrive\\Documents\\UFRJ\\git\\projeto_lupus\\src\\bases_utils.py'>

In [47]:
df_pacientes.shape

(12020493, 8)

In [30]:
cols = ['id_paciente', 'co_procedimento_principal', 'no_procedimento_principal',
            'co_procedimento_secundario', 'no_procedimento_secundario', 'cid_principal', 'cid_secundario']

pac_dict = {}
for id in tqdm(pacientes_sem_lupus):
    dfp = df_pacientes.loc[df_pacientes['id_paciente']==id, cols]
    dfr = bu.verifica_procedimentos_paciente(dfp, ['no_procedimento_principal', 'no_procedimento_secundario'])
    #print(dfr.loc[:,'procedimento_p1':].sum())
    pac_dict[id] = dfr.loc[:,'procedimento_p1':].sum().to_dict()  

df_pacientes_sem_lupus = pd.DataFrame.from_dict(pac_dict).T

100%|██████████| 44/44 [00:28<00:00,  1.52it/s]


In [31]:
dft = pd.DataFrame()
for col in df_pacientes_sem_lupus.columns:
    dft[col] = df_pacientes_sem_lupus[col]>0

In [32]:
dft.mean()

procedimento_p1     0.000000
procedimento_p2     0.659091
procedimento_p3     0.431818
procedimento_p4     0.840909
procedimento_cid    0.136364
dtype: float64

In [53]:
dft[~dft['procedimento_cid']].shape

(38, 5)

In [34]:
dft[dft['procedimento_cid']] # Os que passaram no cid não passaram em nenhun outro procedimento ao que parece

,procedimento_p1,procedimento_p2,procedimento_p3,procedimento_p4,procedimento_cid
1870731500,False,False,False,False,True
2266710300,False,False,False,False,True
5572004900,False,False,False,False,True
22849263200,False,False,False,False,True
25467864600,False,False,False,False,True
27102790700,False,False,False,False,True


In [41]:
df_pacientes.set_index('id_paciente', inplace=True)